# Recommendation System

## Business Understanding

The analysis below is for a custom recommendation system for songs.  The data chosen for training/testing came from the [Million Song Dataset Challenge](https://www.kaggle.com/c/msdchallenge#description). The dataset was originally provided by The Echo Nest and made available by [Columbia University](https://labrosa.ee.columbia.edu/millionsong/) with the puprose encouraging research in machine learning.  The predictive modeling will be done to determine songs the user would like to listen to based on the current song playing.  

## Data Understanding

The analysis will involve two datasets.   The song _usage_ data and the song _information_ data.

The first is the Taste Profile subset.   The data includes a unique hash value for each user along with the song identification number and the number of times the user has played that particular song.  The data was not gathered over a specified time.  Rather the data is dependent upon the user having a minimum of 10 unique songs.  

The second dataset is the song data which provides details about the song.  The usage data only contains the song identification number while the song data provides the song title, artist, release, and year.  

### Downloading the usage and song data

In [1]:
%%capture

import graphlab as gl
import graphlab.aggregate as agg
from matplotlib import pyplot as plt

gl.canvas.set_target('ipynb')

usage_data = gl.SFrame.read_csv("../data/kaggle_visible_evaluation_triplets.txt",
                                header=False,
                                delimiter='\t',
                                column_type_hints={'X3':int})

songs = gl.SFrame.read_csv("../data/song_data.csv")

### Change data labels to be human readable

In [2]:
%%capture
print(usage_data.rename({'X1':'user', 'X2': 'song_id', 'X3': 'plays'}))

The usage data has three fields (`user`, `song_id`, and `plays`) which describes how many times (`play`) a `song` is listened to by a `user`.

In [3]:
usage_data.head(4)

user,song_id,plays
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d ...,SOBONKR12A58A7A7E0,1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d ...,SOEGIYH12A6D4FC0E3,1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d ...,SOFLJQZ12A6D4FADA6,1
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d ...,SOHTKMO12AB01843B0,1


The song data has five fields (`song_id`, `title`, `release`, `artist_name`, and `year`.

In [4]:
songs.head(4)

song_id,title,release,artist_name,year
SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003


To enhance the `usage_data` records details from `songs` data will be included.   The inclusion of the details will allow for subjective review as well as expand the analysis from song usage to artist usage.  A merge on the `usage_data` and `songs` datasets will provide the needed details.

In [6]:
ud_df = usage_data.to_dataframe()
song_df = songs.to_dataframe()

new_df = ud_df.merge(song_df, how='left', left_on='song_id', right_on='song_id')

combo_songs = gl.SFrame(new_df)
combo_songs['song_name'] = combo_songs['title'] + ' (' + combo_songs['artist_name'] +')'

combo_songs['user', 'title', 'artist_name', 'plays', 'song_name'].head(4)

Columns:
	user	str
	title	str
	artist_name	str
	plays	int
	song_name	str

Rows: 4

Data:
+-------------------------------+-------------------------------+
|              user             |             title             |
+-------------------------------+-------------------------------+
| fd50c4007b68a3737fe052d5a4... |         You're The One        |
| fd50c4007b68a3737fe052d5a4... | Horn Concerto No. 4 in E f... |
| fd50c4007b68a3737fe052d5a4... |            Tive Sim           |
| fd50c4007b68a3737fe052d5a4... | Catch You Baby (Steve Pitr... |
+-------------------------------+-------------------------------+
+-------------------------------+-------+--------------------------------+
|          artist_name          | plays |           song_name            |
+-------------------------------+-------+--------------------------------+
|         Dwight Yoakam         |   1   | You're The One (Dwight Yoakam) |
| Barry Tuckwell/Academy of ... |   1   | Horn Concerto No. 4 in E f...  |
|            Cartola            |   1   |       Tive Sim (Cartola)       |
|         Lonnie Gordon         |   1   | Catch You Baby (Steve Pitr...  |
+-------------------------------+-------+--------------------------------+
[4 rows x 5 columns]

Collaborative filtering based on song and user is the primary objective.  In addition, the user and artist data will be analyzed in the same fashion to determine the preferred method.   Both models will be trained, tested, and compared.  

In [9]:
df_artist = new_df.groupby(by=['user','artist_name'])[['plays']].sum().reset_index()

artist = gl.SFrame(df_artist)
artist.head(4)

user,artist_name,plays
00007a02388c208ea7176479f6ae06f8224355b3 ...,Dredg,3
00007a02388c208ea7176479f6ae06f8224355b3 ...,Local H,1
00007a02388c208ea7176479f6ae06f8224355b3 ...,M83,2
00007a02388c208ea7176479f6ae06f8224355b3 ...,No Doubt,1


### Data Visualization

The dataset being utilized for the project is significant in size.   Due to the size, the visuals can be challenging to present.  To provide an image of the number of songs and the number of plays, each entry was categorized based on the number of plays.  The pie chart below shows the percentage of songs in each category.   

In [7]:
#Create the category function
def f(row):
    if row['plays'] <= 1:
        val = ' 0-1'
    elif row['plays'] <= 5:
        val = ' 2-5'
    elif row['plays'] <= 10:
        val =  ' 6-10'
    elif row['plays'] <= 50:
        val =  '11-50'
    elif row['plays'] <= 75:
        val =  '51-75'
    elif row['plays'] >75:
        val =  'Greater than 75'
    return val

#Apply the category function 
new_df['category'] = new_df.apply(f, axis=1)

#Confirm the new category added
new_df.head(3)

,user,song_id,plays,title,release,artist_name,year,category
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,You're The One,If There Was A Way,Dwight Yoakam,1990,0-1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,Horn Concerto No. 4 in E flat K495: II. Romanc...,Mozart - Eine kleine Nachtmusik,Barry Tuckwell/Academy of St Martin-in-the-Fie...,0,0-1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,Tive Sim,Nova Bis-Cartola,Cartola,1974,0-1


In [8]:
#Group by categories

pie_chart = new_df.groupby(by=['category'])[['plays']].count().reset_index()

pie_chart.head()

,category,plays
0,0-1,862354
1,2-5,451292
2,6-10,100857
3,11-50,72879
4,51-75,2699


In [9]:
#Plot
plt.pie(pie_chart['plays'], labels=pie_chart['category'], autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

([<matplotlib.patches.Wedge at 0x12113a910>,
  <matplotlib.text.Text at 0x12119d750>])

In [ ]:
#Apply the category function 
df_artist['category'] = df_artist.apply(f, axis=1)
                                  
art_pc = df_artist.groupby(by=['category'])[['plays']].count().reset_index()
art_pc.head()

plt.pie(art_pc['plays'], labels=art_pc['category'], autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

With the song data, over half of the information is based on a song being played a single time.  All data will remain in the analysis; however, the model will be chosen to consider the number of times the song was played.

### GraphLab Canvas

In [23]:
combo_songs.show()

In [24]:
artist.show()

Executing the `graphlab show` commands above will provide summary information about the data in a local browser using GraphLab Canvas.   

The song data includes over 110,000 users listening to over 166,433 unique songs.   The data includes 138,499 unqiue song titles.   The inference is 27,934 songs have the same title although they are different songs.   Analysis will need to be done on the `song_id` rather than the `title`.   Songs were played between 1 times and 923 times.  Both the song data as well as the artist data contain 28,266 unique artists.   Neither dataset has missing values.  The year has some missing values that are populated with 0.  Detail on the missing years can be found in the GraphLab Canvas.   Due to 20% of the year data missing, analysis will not include the year informaiton.  

Since the user identification is based on a user ID, the possiblities of a single person having multiple user IDs or a single user ID being used by multiple individuals cannot be eliminated.   Since the sharing of user IDs is common with families, our analysis will proceed with caution.  

## Train and Adjust parameters

In [12]:
combo_songs.head(3)

Columns:
	user	str
	song_id	str
	plays	int
	title	str
	release	str
	artist_name	str
	year	int
	song_name	str

Rows: 3

Data:
+-------------------------------+--------------------+-------+
|              user             |      song_id       | plays |
+-------------------------------+--------------------+-------+
| fd50c4007b68a3737fe052d5a4... | SOBONKR12A58A7A7E0 |   1   |
| fd50c4007b68a3737fe052d5a4... | SOEGIYH12A6D4FC0E3 |   1   |
| fd50c4007b68a3737fe052d5a4... | SOFLJQZ12A6D4FADA6 |   1   |
+-------------------------------+--------------------+-------+
+-------------------------------+-------------------------------+
|             title             |            release            |
+-------------------------------+-------------------------------+
|         You're The One        |       If There Was A Way      |
| Horn Concerto No. 4 in E f... | Mozart - Eine kleine Nacht... |
|            Tive Sim           |        Nova Bis-Cartola       |
+-------------------------------+-------------------------------+
+-------------------------------+------+--------------------------------+
|          artist_name          | year |           song_name            |
+-------------------------------+------+--------------------------------+
|         Dwight Yoakam         | 1990 | You're The One (Dwight Yoakam) |
| Barry Tuckwell/Academy of ... |  0   | Horn Concerto No. 4 in E f...  |
|            Cartola            | 1974 |       Tive Sim (Cartola)       |
+-------------------------------+------+--------------------------------+
[3 rows x 8 columns]

#### Song Recommendation
The following is the training and testing of the song usage data.  The analysis includes both the user and the song information.    

A recommender based on the plays from users will be created.  The recommender will suggest 5 songs for each user based on the feedback from other users.

In [13]:
train_song, test_song = gl.recommender.util.random_split_by_user(combo_songs,
                                                      user_id="user", item_id="song_id",
                                                      item_test_proportion=0.2)

song_rec = gl.recommender.item_similarity_recommender.create(train_song,
                                                             user_id="user",
                                                             item_id="song_id",
                                                             target="plays",
                                                             only_top_k=5,
                                                             similarity_type="cosine")

rmse_results = song_rec.evaluate(test_song)

print rmse_results.viewkeys()

Recsys training: model = item_similarity

Warning: Ignoring columns title, release, artist_name, year, song_name;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1489352 observations with 110000 users and 163095 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.21949s


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.00463499420626 | 0.00225955967555 |
|   2    | 0.00463499420626 | 0.00414252607184 |
|   3    | 0.00502124372345 | 0.00643243392374 |
|   4    | 0.00463499420626 | 0.0074808254704  |
|   5    | 0.00393974507532 | 0.00786707498758 |
|   6    | 0.00347624565469 | 0.00815676212548 |
|   7    | 0.00430392319152 | 0.0116054185289  |
|   8    | 0.00420046349942 | 0.0124551674668  |
|   9    | 0.00424874468907 |  0.015517574353  |
|   10   | 0.00428736964079 | 0.0161079135876  |
+--------+------------------+------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 6.278726095375254)

Per User RMSE (best)
+-------------------------------+-------+----------------+
|              user             | count |      rmse      |
+-------------------------------+-------+----------------+
|

In [14]:
rmse_results['rmse_overall']

6.278726095375254

In [16]:
rmse_results['rmse_by_user'].head(4)

user,count,rmse
c174d59c27220c94999bf67cd98fd3eca67f7ab0 ...,4,1.0
1be569aefb976bae9aa6a29af47db2e8d666dcd2 ...,9,4.20317340431
e8f5730a122c64e706a3fb9b47dc33d30d1e4c18 ...,8,3.42336455287
9b9fb6acabe8bdde3012ddc21626c435e62bc077 ...,7,1.88982236505


In [17]:
rmse_results['rmse_by_item'].head(4)

song_id,count,rmse
SOEKEWC12A8C132A59,1,1.0
SOQIPYO12AAF3B5B1D,1,10.0
SOYIOZB12A58A797FC,1,4.0
SOSZAST12A6D4F6245,1,2.0


In [25]:
train_song, test_song = gl.recommender.util.random_split_by_user(combo_songs,
                                                      user_id="user", item_id="song_id",
                                                      item_test_proportion=0.2)

song_rec = gl.recommender.item_similarity_recommender.create(train_song,
                                                             user_id="user",
                                                             item_id="song_id",
                                                             target="plays",
                                                             only_top_k=5,
                                                             similarity_type="pearson")

rmse_results = song_rec.evaluate(test_song)

rmse_results.viewkeys()

Recsys training: model = item_similarity

Warning: Ignoring columns title, release, artist_name, year, song_name;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1489352 observations with 110000 users and 163095 items.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.39432s


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 5.471660027577793)

Per User RMSE (best)
+-------------------------------+-------+------+
|              user             | count | rmse |
+-------------------------------+-------+------+
| bb560bb8852897da731e52705e... |   1   | 0.0  |
+-------------------------------+-------+------+
[1 rows x 3 columns]


Per User

dict_keys(['rmse_by_user', 'precision_recall_overall', 'rmse_by_item', 'precision_recall_by_user', 'rmse_overall'])

In [19]:
rmse_results['rmse_overall']

5.4716600275776

In [21]:
rmse_results['rmse_by_user'].head(4)

user,count,rmse
c174d59c27220c94999bf67cd98fd3eca67f7ab0 ...,4,1.63401832999
1be569aefb976bae9aa6a29af47db2e8d666dcd2 ...,9,2.04651934683
e8f5730a122c64e706a3fb9b47dc33d30d1e4c18 ...,8,3.12925062706
9b9fb6acabe8bdde3012ddc21626c435e62bc077 ...,7,1.9644652305


In [22]:
rmse_results['rmse_by_item'].head(4)

song_id,count,rmse
SOEKEWC12A8C132A59,1,2.75925925926
SOQIPYO12AAF3B5B1D,1,6.83146067416
SOYIOZB12A58A797FC,1,1.30303030303
SOSZAST12A6D4F6245,1,0.516129032258


The RMSE (root mean square error) is evaluated for both the cosine and pearson model for the item similarity recommender.  The pearson model is the preferred model with the lower RMSE (5.47 compared to 6.28).

#### Artist Recommendation
Music recommendations may be offered in multiple forms.  In addition to the recommendation of songs, the analysis includes the recommendation of music by artist.  The data was trained as tested the same as the song data to offere comparison of the two collaborative filtering results.   

In [26]:
train_art, test_art = gl.recommender.util.random_split_by_user(artist,
                                                      user_id="user", item_id="artist_name",
                                                      item_test_proportion=0.2)

art_rec = gl.recommender.item_similarity_recommender.create(train_art,
                                                             user_id="user",
                                                             item_id="artist_name",
                                                             target="plays",
                                                             only_top_k=5,
                                                             similarity_type="cosine")

rmse_results = art_rec.evaluate(test_art)

rmse_results.viewkeys()

Recsys training: model = item_similarity

Preparing data set.

Data has 1131349 observations with 109996 users and 28359 items.

Data prepared in: 1.21072s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 8.18355s


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.0024154589372  | 0.00144927536232 |
|   2    | 0.0024154589372  | 0.00305958132045 |
|   3    | 0.00201288244767 | 0.00346215780998 |
|   4    | 0.00332125603865 | 0.00678341384863 |
|   5    | 0.00434782608696 | 0.0113123993559  |
|   6    | 0.00402576489533 | 0.0122181964573  |
|   7    | 0.0036231884058  | 0.0126207729469  |
|   8    | 0.00347222222222 | 0.0141304347826  |
|   9    | 0.00322061191626 | 0.0143719806763  |
|   10   | 0.00326086956522 | 0.0163345410628  |
+--------+------------------+------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 13.99781300253604)

Per User RMSE (best)
+-------------------------------+-------+----------------+
|              user             | count |      rmse      |
+-------------------------------+-------+----------------+
|

dict_keys(['rmse_by_user', 'precision_recall_overall', 'rmse_by_item', 'precision_recall_by_user', 'rmse_overall'])

In [27]:
rmse_results['rmse_overall']

13.99781300253604

In [29]:
rmse_results['rmse_by_user'].head(4)

user,count,rmse
c174d59c27220c94999bf67cd98fd3eca67f7ab0 ...,4,1.0
1be569aefb976bae9aa6a29af47db2e8d666dcd2 ...,4,10.4642247682
cfa3a9fb3d859ace61a1e05fd5e51f3cef9cd3e6 ...,4,8.09320702812
e8f5730a122c64e706a3fb9b47dc33d30d1e4c18 ...,7,9.53904623816


In [30]:
rmse_results['rmse_by_item'].head(4)

artist_name,count,rmse
Spoon,2,6.0827625303
Scooter,1,5.0
Pru,1,1.0
Dragonland,1,1.0


In [31]:
train_art, test_art = gl.recommender.util.random_split_by_user(artist,
                                                      user_id="user", item_id="artist_name",
                                                      item_test_proportion=0.2)

art_rec = gl.recommender.item_similarity_recommender.create(train_art,
                                                             user_id="user",
                                                             item_id="artist_name",
                                                             target="plays",
                                                             only_top_k=5,
                                                             similarity_type="pearson")

rmse_results = art_rec.evaluate(test_art)

print rmse_results.viewkeys()

Recsys training: model = item_similarity

Preparing data set.

Data has 1131349 observations with 109996 users and 28359 items.

Data prepared in: 1.18551s

Training model from provided data.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 7.30669s


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 13.234525217052376)

Per User RMSE (best)
+-------------------------------+-------+------+
|              user             | count | rmse |
+-------------------------------+-------+------+
| 7fd613423ff2305c23ab30d5f9... |   1   | 0.0  |
+-------------------------------+-------+------+
[1 rows x 3 columns]


Per Use

In [32]:
rmse_results['rmse_overall']

13.234525217052376

In [33]:
rmse_results['rmse_by_user'].head(4)

user,count,rmse
c174d59c27220c94999bf67cd98fd3eca67f7ab0 ...,4,2.93748076643
1be569aefb976bae9aa6a29af47db2e8d666dcd2 ...,4,7.23482442001
cfa3a9fb3d859ace61a1e05fd5e51f3cef9cd3e6 ...,4,6.13846481143
e8f5730a122c64e706a3fb9b47dc33d30d1e4c18 ...,7,7.54462252268


In [34]:
rmse_results['rmse_by_item'].head(4)

artist_name,count,rmse
Spoon,2,2.17400503742
Scooter,1,0.283068783069
Pru,1,0.45
Dragonland,1,0.533333333333


## Evaluate and Compare

The recommendation models were run for the artist prediction to offer a comparison.   Although it may be expected that the recommendation model would be better with the same amount plays data yet fewer recommendations; however, that was incorrect.  Both models for the artist prediction were significantly higher than the song recommendation counterpart.   

The model for song recommendation using the pearsons similarity type will be used for the remainder of the analysis.  

## Visualize Results

In [35]:
#Preferred method selected above
train_song, test_song = gl.recommender.util.random_split_by_user(combo_songs,
                                                      user_id="user", item_id="song_id",
                                                      item_test_proportion=0.2)

song_rec = gl.recommender.item_similarity_recommender.create(train_song,
                                                             user_id="user",
                                                             item_id="song_id",
                                                             target="plays",
                                                             only_top_k=5,
                                                             similarity_type="pearson")

rmse_results = song_rec.evaluate(test_song)

print rmse_results.viewkeys()

Recsys training: model = item_similarity

Warning: Ignoring columns title, release, artist_name, year, song_name;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1489352 observations with 110000 users and 163095 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.04957s


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 5.471660027577793)

Per User RMSE (best)
+-------------------------------+-------+------+
|              user             | count | rmse |
+-------------------------------+-------+------+
| bb560bb8852897da731e52705e... |   1   | 0.0  |
+-------------------------------+-------+------+
[1 rows x 3 columns]


Per User

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

rmsevals = rmse_results['rmse_by_item']['rmse']

plt.hist(rmsevals, bins=20)
plt.title('RMSE by Song')
plt.show()

The RMSE of most songs is relatively low.   However, we can see a few songs with very high RMSE or errors in the prediction from the training set to the testing set.   

# To do:  find way to list highest RMSE songs

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

rmsevals = rmse_results['rmse_by_user']['rmse']
rmsevals = rmsevals[rmse_results['rmse_by_user']['count']>10]


plt.hist(rmsevals, bins=20)
plt.title('RMSE by User')
plt.show()

A few users have high RMSE.   As mentioned in the early analysis of the data, errors for the prediction of certain users may be related to multiple family members accessing the music using the same user ID.  In additon, the high RMSE may be due to eclectic music plays.

### Ranking Factorization Recommender
The ranking factorication recommender learns factors from the user and item.  This model of prediction is preferred for rankings.  Since the plays data is similar to ranking, we will explore the effectiveness of this model.   This model is also know as cross validating collaborative filtering.   The model is very time consuming to process with a large dataset. 

In [36]:
recl = gl.recommender.ranking_factorization_recommender.create(train_song,
                                                             user_id="user",
                                                             item_id="song_id",
                                                             target="plays")

rmse_results = recl.evaluate(test_song)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 1489352 observations with 110000 users and 163095 items.

Data prepared in: 3.72523s

Training ranking_factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 44.3193

Final training RMSE: 5.61823


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |  0.00115874855156 | 0.000289687137891 |
|   2    | 0.000579374275782 | 0.000289687137891 |
|   3    | 0.000386249517188 | 0.000289687137891 |
|   4    | 0.000579374275782 | 0.000579374275782 |
|   5    | 0.000463499420626 | 0.000579374275782 |
|   6    | 0.000386249517188 | 0.000579374275782 |
|   7    | 0.000331071014733 | 0.000579374275782 |
|   8    | 0.000289687137891 | 0.000579374275782 |
|   9    | 0.000257499678125 | 0.000579374275782 |
|   10   | 0.000463499420626 | 0.000939873825158 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 5.9122153029011795)

Per User RMSE (best)
+-------------------------------+-------+-----------------+
|              user             | count |       rmse      |
+----------------------------

In [37]:
rmse_results['rmse_overall']

5.9122153029011795

The cross validation model increased the RMSE from 5.47 to 7.34.  

In [38]:
rmse_results['precision_recall_by_user'].groupby('cutoff',[agg.AVG('precision'),agg.STD('precision'),agg.AVG('recall')])

cutoff,Avg of precision,Stdv of precision,Avg of recall
16,0.000434530706837,0.00519320250348,0.00180893523883
10,0.000463499420626,0.00679228314705,0.000939873825158
36,0.000708124114845,0.00457786705229,0.0087356765804
26,0.000668508779749,0.00502642737494,0.00542036822454
41,0.000678291835062,0.00417878755405,0.0104737994078
3,0.000386249517188,0.0113402226774,0.000289687137891
1,0.00115874855156,0.0340206680322,0.000289687137891
6,0.000386249517188,0.00801409575946,0.000579374275782
11,0.00042136310966,0.00617480286095,0.000939873825158
2,0.000579374275782,0.0170103340161,0.000289687137891


The aggregate of the precision and recall are evaluated for this model.   Precision is the measurement of not predicting falsely.  Recall is the measurement of correct predictions.  Accuracy is the measure the fraction of prediting correct.  

### Determine correct number of latent factors

Since the RMSE decreased with the cross validation, the model will be slightly adjusted from the default to analyze again with the number latent factors clearly designated.  Before determining the correct number of latent factors, models will be run for 4 different possible latent factors.  

In [39]:
params = {'user_id': 'user',
         'item_id': 'song_id',
         'target': 'plays',
         'num_factors': [4, 10, 20],
         'regularization': [0.001] ,
         'linear_regularization': [0.001]}

job = gl.model_parameter_search.create( (train_song,test_song),
                                      gl.recommender.ranking_factorization_recommender.create,
                                      params,
                                      max_models=3,
                                      environment=None)


In [ ]:
job.get_status()

In [ ]:
job_result = job.get_results()

job_result.head()

In [ ]:
bst_prms = job.get_best_params()
bst_prms

In [ ]:
models = job.get_models()
models

In [ ]:
comparisonstruct = gl.compare(test_song,models)
gl.show_comparison(comparisonstruct,models)

In [ ]:
models[1]

# Based on the results of the above - manually change the num_factors to that number for the latent factors.

In [ ]:
recl = gl.recommender.ranking_factorization_recommender.create(train_song,
                                                             user_id="user",
                                                             item_id="song_id",
                                                             target="plays",
                                                              num_factors=10, 
                                                              regularization=1e-02,
                                                              linear_regularization = 1e-3)

rmse_results = recl.evaluate(test_song)

In [ ]:
rmse_results['rmse_overall']

In [ ]:
rmse_results['precision_recall_by_user'].groupby('cutoff',[agg.AVG('precision'),agg.STD('precision'),agg.AVG('recall')])

The comparison between the models

In [ ]:
comparisonstruct = gl.compare(test_song, [song_rec, recl])

In [ ]:
comparisonstruct = gl.compare(test_song, [song_rec, recl])

In [ ]:
gl.show_comparison(comparisonstruct,[song_rec, recl])

## Summarize the Ramifications

## Deployment

Turi has in depth documentation on how to deploy a GraphLab model. Specifically, there seems to be three ways to make a deployment:

 - Turi Distributed
 - Turi Predictive Services
 - Simply running `predict` against your chosen model
 
 
[Turi Distributed](https://turi.com/learn/userguide/deployment/pipeline-introduction.html) provides a way to distribute to ec2 or hadoop clusters. Turi Distributed also adds some management tools to help with your production environment.

[Turi Predictive Services](https://turi.com/learn/userguide/deployment/pred-intro.html) offers another service to deploy your models. This way may be considered to be more robust as much of the deployment process and management is completely automated.

If the production environment would be on a local environment, you could just load your model locally and run predictions as needed.


## Exceptional Work

We were really excited to see this project in action. We tried all three different ways to deploy our model. The first two "automated" ways to deploy threw errors when we got to the final steps due to some S3 error.

We attempted to create our own small environment. The working example can be seen below:

In [ ]:
from IPython.display import IFrame
IFrame('http://ec2-52-53-178-233.us-west-1.compute.amazonaws.com:3000/', width=700, height=220)

As you can see, if you select a song title, another song will be recommended based on your choice. This project includes having a python script that polls for a `song_id` as its stdin, which then returns a list of recommendations. A nodejs server sits above the python script to provide a fast, lightweight web server to serve the recommendations. The full source code for this project can be found [here](https://github.com/kjprice/graph-lab-demo).

## References

Reference for the dataset:
@INPROCEEDINGS{Bertin-Mahieux2011,
  author = {Thierry Bertin-Mahieux and Daniel P.W. Ellis and Brian Whitman and Paul Lamere},
  title = {The Million Song Dataset},
  booktitle = {{Proceedings of the 12th International Conference on Music Information
	Retrieval ({ISMIR} 2011)}},
  year = {2011},
  owner = {thierry},
  timestamp = {2010.03.07}
  